In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa
import re
import findspark
findspark.init('/home/dave/spark-2.4.1-bin-hadoop2.7/')
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import *
import sys
import os
from IPython.display import display, HTML
from pyspark.sql.functions import pandas_udf, PandasUDFType
#from pyspark.sql.functions import unix_timestamp
#from pyspark.sql.functions import from_unixtime



datafile = "/home/dave/Documents/CS691/proj/data/snotel/SNOTEL_518_HEAVENLY_VALLEY"
spark = SparkSession.builder.appName("proj_exp").getOrCreate()

df = spark.read.text(datafile)
df= df.filter(~ df.value.rlike('#'))
df= df.filter(~ df.value.rlike('Date'))
  
#split DF into Cols: Date,Station Name,Station Id,Snow Water Equivalent (in),Change In Snow Water Equivalent (in),
#Snow Depth (in),Change In Snow Depth (in),Air Temperature Observed (degF),Barometric Pressure (inch_Hg),Dew Point 
#Temperature (degF),Wind Direction Observed (degree),Wind Speed Observed (mph)
split_col = pyspark.sql.functions.split(df['value'], ',')

df = df.withColumn('Date', split_col.getItem(0))
df = df.withColumn('Station Name', split_col.getItem(1))
df = df.withColumn('sid', split_col.getItem(2))
df = df.withColumn('swe', split_col.getItem(3))
df = df.withColumn('dswe', split_col.getItem(4))
df = df.withColumn('sd', split_col.getItem(5))
df = df.withColumn('dsd', split_col.getItem(6))
df = df.withColumn('at', split_col.getItem(7))

#create delta air temp
w = Window().partitionBy().orderBy(F.col("Date"))
df = df.select("*", F.lag("at").over(w).alias("prev_at"))
df = df.withColumn("dat", F.when(F.isnull(df.at - df.prev_at), 0)
                              .otherwise(df.at - df.prev_at))
df = df.select('Date', 'Station Name', 'sid', 'swe', 'dswe', 'sd', 'dsd', 'at', 'dat')
cr = df.select('Date', 'sid')

#split date and time
split_col = pyspark.sql.functions.split(df['Date'], ' ')
df = df.withColumn('date_only', split_col.getItem(0))
df = df.withColumn('time_only', split_col.getItem(1))
df = df.select('Date', 'date_only', 'time_only', 'Station Name', 'sid', 'swe', 'dswe', 'sd', 'dsd', 'at', 'dat')
#sid- station id
#swe- snow water equivalent
#dswe- delta swe
#sd- snow depth
#dsd- delta sd
#at- air temp
#dat- delta air temp

#make compass rose
#not currently used- maintaining for future use
#compass rose
# array index
#---0----------1---------2--------3----------4--------5---------6--------7--------
#---west---northwest---north---northeast---east---southeast---south---southwest---

def ones(x):
    return [1,1,1,1,1,1,1,1]
    
ones_udf_array = F.udf(lambda z: ones(z), ArrayType(IntegerType()))
        
#cr = cr.select('*', ones_udf_array('sid').alias('below_treeline'))
#cr = cr.select('*', ones_udf_array('sid').alias('near_treeline'))
#cr = cr.select('*', ones_udf_array('sid').alias('above_treeline'))

def sd(x):
    return [1,1,1,1,1,1,1,1]
    
sd_udf_array = F.udf(lambda z: sd(z), ArrayType(IntegerType()))
#cr = cr.select('*', ones_udf_array('sid').alias('below_treeline'))

#join input and output tables for processing (remove dup cols)
df = df.select('Date', 'date_only', 'time_only', 'Station Name', 'swe', 'dswe', 'sd', 'dsd', 'at', 'dat')
sc = df.join(cr, ["Date"])

#unix date format timestamp
sc = sc.withColumn('timestamp',F.col('Date').cast("timestamp"))
sc = sc.select('timestamp','date_only','time_only','swe', 'dswe', 'dsd').orderBy("timestamp")

#add row id for later join
sc = sc.withColumn("row_num", F.row_number().over(Window.orderBy("timestamp")))

#pandas udf processing

x_columns = ['time_only', 'dsd', 'row_num']
schema = sc.select("date_only", *x_columns).schema
sc = sc.select("date_only", *x_columns)

def snowfall(x):
    try:
        if int(x) >= 4 :
            return "1"
        elif int(x) >= 2:
            return "0.5"
        else:
            return "0"
    except: 
        return "0"

#udf to assess av risk from delta snow depth
# Input/output are both a pandas.DataFrame

@pandas_udf(schema, F.PandasUDFType.GROUPED_MAP)
def dsd_calc_udf(pdf):
    #sort_by_dsd = pdf.sort_values(by=['dsd'],ascending = False)
    pdf.dsd = pdf.dsd.apply(snowfall)
    return pdf
    #return pd.DataFrame([[group_key] + [model.params[i] for i in   x_columns]], columns=[group_column] + x_columns)

#result of avalanche risk calc, need to cumsum across time with 2 day window
av_dsd = sc.groupby("date_only").apply(dsd_calc_udf)

##create delta snow water equiv and process risk vector

sc = df.join(cr, ["Date"])
x_columns = ['Date','time_only','swe', 'dswe', 'dsd']
schema = sc.select("date_only", *x_columns).schema
sc = sc.select("date_only", *x_columns)
sc = sc.filter(sc.date_only== "2019-02-15")

sc = sc.withColumn("dswe", F.abs(sc["dswe"].cast(DoubleType())))

#cast date to timesampe and order
sc = sc.withColumn('timestamp',F.col('Date').cast("timestamp"))
sc = sc.select('timestamp','date_only','time_only','swe', 'dswe', 'dsd').orderBy("timestamp")

#add row id for ranging
sc = sc.withColumn("row_num", F.row_number().over(Window.orderBy("timestamp")))

#cum sum dswe over window
window = Window.orderBy("row_num").rangeBetween(-24, 0)
sc = sc.withColumn('cum_sum_dswe', F.sum('dswe').over(window))

#cum sum dswe over window
window = Window.orderBy("row_num").rangeBetween(-48, 0)
av_dsd = av_dsd.withColumn('cum_sum_dsd', F.sum('dsd').over(window))

av_dsd = av_dsd.select( 'row_num','cum_sum_dsd','dsd')

sc = sc.select('timestamp','row_num', 'cum_sum_dswe')
sc = sc.join(av_dsd, ["row_num"])

#calculate wet loose temperature risk

df = df.withColumn('at',F.col('at').cast(IntegerType()))
df = df.withColumn('wl_av',F.when(df.at > 40, 1).otherwise(0))

df.printSchema()
df.show()

sc.printSchema()
sc.show()

spark.stop()

root
 |-- Date: string (nullable = true)
 |-- date_only: string (nullable = true)
 |-- time_only: string (nullable = true)
 |-- Station Name: string (nullable = true)
 |-- swe: string (nullable = true)
 |-- dswe: string (nullable = true)
 |-- sd: string (nullable = true)
 |-- dsd: string (nullable = true)
 |-- at: integer (nullable = true)
 |-- dat: double (nullable = true)
 |-- wl_av: integer (nullable = false)

+----------------+----------+---------+---------------+----+----+---+---+---+----+-----+
|            Date| date_only|time_only|   Station Name| swe|dswe| sd|dsd| at| dat|wl_av|
+----------------+----------+---------+---------------+----+----+---+---+---+----+-----+
|2009-04-01 00:00|2009-04-01|    00:00|Heavenly Valley|18.6| 0.0| 50|  0| 33| 0.0|    0|
|2009-04-01 01:00|2009-04-01|    01:00|Heavenly Valley|18.5|-0.1| 49| -1| 32|-1.0|    0|
|2009-04-01 02:00|2009-04-01|    02:00|Heavenly Valley|18.5| 0.0| 49|  0| 32| 0.0|    0|
|2009-04-01 03:00|2009-04-01|    03:00|Heavenly V

In [ ]:
r